# Create Combined All Data with NLP Features

In [45]:
# Packages
import pandas as pd
import os
import shutil

## Load NLP Features

In [46]:
# Read '~/Box/STAT 222 Capstone/Intermediate Data/Calls/all_NLP_features.parquet'
# Limit columsn to ticker, earnings_call_date, num_transparency, gf_score, readability, word_count, num_questions, pos_score
all_NLP_features = pd.read_parquet('~/Box/STAT 222 Capstone/Intermediate Data/Calls/all_NLP_features.parquet', columns = ['ticker', 'earnings_call_date', 'num_transparency', 'gf_score', 'readability', 'word_count', 'num_questions', 'pos_score'])
print(all_NLP_features.head())
print('length of all_NLP_features:', len(all_NLP_features))

  ticker earnings_call_date  num_transparency   gf_score  readability  \
0   AAPL         2014-04-23              0.11  12.369779    12.369779   
1   AAPL         2014-07-22              0.10  12.781526    12.781526   
2   AAPL         2014-10-20              0.12  12.537288    12.537288   
3   AAPL         2015-01-27              0.11  11.759278    11.759278   
4   AAPL         2015-04-27              0.11  12.013334    12.013334   

   word_count  num_questions  pos_score  
0        8869             21   0.413608  
1        7587             29   0.340539  
2        9399             26   0.435410  
3        9075             42   0.340825  
4        8548             33   0.482874  
length of all_NLP_features: 7316


In [47]:
# Read '~/Box/STAT 222 Capstone/Intermediate Data/Calls/word_tone.parquet'
word_tone = pd.read_parquet('~/Box/STAT 222 Capstone/Intermediate Data/Calls/word_tone.parquet')
print(word_tone.head())
print('length of word_tone:', len(word_tone))

   Positiv  Negativ  Strong  Weak  Active  Passive  Ovrst  Undrst        PN  \
0      373       48     701    55     573      189    458     179  7.770833   
1      298       54     641    42     495      186    364     131  5.518519   
2      353       66     733    46     656      199    465     152  5.348485   
3      326       83     714    88     591      208    468     151  3.927711   
4      315       60     640    70     565      214    415     135  5.250000   

          SW        AP        OU     TONE1  
0  12.745455  3.031746  2.558659  3.452048  
1  15.261905  2.661290  2.778626  3.188264  
2  15.934783  3.296482  3.059211  3.681858  
3   8.113636  2.841346  3.099338  1.307366  
4   9.142857  2.640187  3.074074  2.025933  
length of word_tone: 7316


In [48]:
# Concatenate all_NLP_features and word_tone
nlp_features_df = pd.concat([all_NLP_features, word_tone], axis = 1)
nlp_features_df

,ticker,earnings_call_date,num_transparency,gf_score,readability,word_count,num_questions,pos_score,Positiv,Negativ,...,Weak,Active,Passive,Ovrst,Undrst,PN,SW,AP,OU,TONE1
0,AAPL,2014-04-23,0.11,12.369779,12.369779,8869,21,0.413608,373,48,...,55,573,189,458,179,7.770833,12.745455,3.031746,2.558659,3.452048
1,AAPL,2014-07-22,0.10,12.781526,12.781526,7587,29,0.340539,298,54,...,42,495,186,364,131,5.518519,15.261905,2.661290,2.778626,3.188264
2,AAPL,2014-10-20,0.12,12.537288,12.537288,9399,26,0.435410,353,66,...,46,656,199,465,152,5.348485,15.934783,3.296482,3.059211,3.681858
3,AAPL,2015-01-27,0.11,11.759278,11.759278,9075,42,0.340825,326,83,...,88,591,208,468,151,3.927711,8.113636,2.841346,3.099338,1.307366
4,AAPL,2015-04-27,0.11,12.013334,12.013334,8548,33,0.482874,315,60,...,70,565,214,415,135,5.250000,9.142857,2.640187,3.074074,2.025933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7311,ZTS,2015-08-04,0.09,15.912200,15.912200,8228,12,0.192155,372,103,...,52,623,214,298,148,3.611650,15.634615,2.911215,2.013514,1.744657
7312,ZTS,2015-11-03,0.14,13.175559,13.175559,10625,41,0.110071,501,133,...,79,737,264,395,222,3.766917,15.848101,2.791667,1.779279,1.596294
7313,ZTS,2016-02-16,0.16,12.896152,12.896152,11455,45,0.176927,517,145,...,73,840,287,469,217,3.565517,17.506849,2.926829,2.161290,2.287146
7314,ZTS,2016-05-04,0.12,12.342619,12.342619,10370,33,0.200915,418,117,...,68,765,253,449,215,3.572650,15.235294,3.023715,2.088372,1.739992


In [49]:
# Check uniqueness on ticker and earnings_call_date
print('unique on ticker and earnings_call_date:')
print(len(nlp_features_df) == len(nlp_features_df[['ticker', 'earnings_call_date']].drop_duplicates()))

unique on ticker and earnings_call_date:
True


## Merge with all data file

In [50]:
# list of files in '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
file_list = [f for f in os.listdir(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates') if f.endswith('.parquet')]
# read in all parquet files
all_data_fixed_quarter_dates = pd.concat([pd.read_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/' + f) for f in file_list])
all_data_fixed_quarter_dates

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector,train_test_80_20
0,AAPL,2014-07-01,2014-04-23,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.480288,4.551851,2014-04-24,None,True,NaN,None,NaN,Information Technology,train
1,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.443623,4.324703,2014-07-23,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology,train
2,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.375916,5.727053,2014-10-27,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology,train
3,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,9,AA,2015-05-28,AA,2014-05-27,...,0.371059,3.992323,2015-01-28,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology,train
4,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,9,AA,2015-08-25,AA,2015-05-28,...,0.386379,5.869492,2015-04-28,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,ZTS,2015-10-01,2015-08-04,BBB,2015-01-30,7,BBB,2015-11-03,BBB,2014-01-31,...,0.120491,3.500267,2015-08-06,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care,train
832,ZTS,2016-01-01,2015-11-03,BBB,2015-11-03,7,BBB,2016-01-22,BBB,2015-01-30,...,0.141041,3.198497,2015-11-05,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care,train
833,ZTS,2016-04-01,2016-02-16,BBB,2016-01-22,7,BBB,2016-12-23,BBB,2015-11-03,...,0.110704,2.782747,2016-02-24,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care,test
834,ZTS,2016-07-01,2016-05-04,BBB,2016-01-22,7,BBB,2016-12-23,BBB,2015-11-03,...,0.136321,3.000131,2016-05-06,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care,train


In [51]:
# Merge
# Convert earnings_call_date to datetime in both
all_data_fixed_quarter_dates['earnings_call_date'] = pd.to_datetime(all_data_fixed_quarter_dates['earnings_call_date'])
nlp_features_df['earnings_call_date'] = pd.to_datetime(nlp_features_df['earnings_call_date'])
all_data_with_NLP_features = pd.merge(all_data_fixed_quarter_dates, nlp_features_df, how = 'left', on = ['ticker', 'earnings_call_date'])
all_data_with_NLP_features

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Weak,Active,Passive,Ovrst,Undrst,PN,SW,AP,OU,TONE1
0,AAPL,2014-07-01,2014-04-23,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,55.0,573.0,189.0,458.0,179.0,7.770833,12.745455,3.031746,2.558659,3.452048
1,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,42.0,495.0,186.0,364.0,131.0,5.518519,15.261905,2.661290,2.778626,3.188264
2,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,46.0,656.0,199.0,465.0,152.0,5.348485,15.934783,3.296482,3.059211,3.681858
3,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,9,AA,2015-05-28,AA,2014-05-27,...,88.0,591.0,208.0,468.0,151.0,3.927711,8.113636,2.841346,3.099338,1.307366
4,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,9,AA,2015-08-25,AA,2015-05-28,...,70.0,565.0,214.0,415.0,135.0,5.250000,9.142857,2.640187,3.074074,2.025933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,ZTS,2015-10-01,2015-08-04,BBB,2015-01-30,7,BBB,2015-11-03,BBB,2014-01-31,...,52.0,623.0,214.0,298.0,148.0,3.611650,15.634615,2.911215,2.013514,1.744657
6679,ZTS,2016-01-01,2015-11-03,BBB,2015-11-03,7,BBB,2016-01-22,BBB,2015-01-30,...,79.0,737.0,264.0,395.0,222.0,3.766917,15.848101,2.791667,1.779279,1.596294
6680,ZTS,2016-04-01,2016-02-16,BBB,2016-01-22,7,BBB,2016-12-23,BBB,2015-11-03,...,73.0,840.0,287.0,469.0,217.0,3.565517,17.506849,2.926829,2.161290,2.287146
6681,ZTS,2016-07-01,2016-05-04,BBB,2016-01-22,7,BBB,2016-12-23,BBB,2015-11-03,...,68.0,765.0,253.0,449.0,215.0,3.572650,15.235294,3.023715,2.088372,1.739992


## Remove extraneous variables

In [52]:
# Remove extraneous variables 'Rating Rank AAA is 10', 'rating_on_previous_fixed_quarter_date AAA is 10', 'readability'
all_data_with_NLP_features = all_data_with_NLP_features.drop(['Rating Rank AAA is 10', 'rating_on_previous_fixed_quarter_date AAA is 10', 'readability'], axis = 1)

## Drop Items Missing Any Required Variables

In [53]:
# Print all columns
for col in all_data_with_NLP_features.columns:
    print(col)
# Output to Excel
all_cols = pd.DataFrame(all_data_with_NLP_features.columns)
all_cols.columns = ['column_name']
all_cols.to_excel('all_data_with_NLP_features_columns_before_drop_of_missing.xlsx', index = False)

ticker
fixed_quarter_date
earnings_call_date
Rating
rating_date
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call_on_fixed_quarter
days_since_rating
for_quarter
for_year
transcript
reportedCurrency
acceptedDate_balance_sheet
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory_balance_sheet
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
otherCurrentLiabilities
totalCurrentLiabilities
longTermDebt
deferredRevenueNonCurrent
deferredTaxLiabilitiesNonCurrent
otherNonCurrentLiabilities
totalNonCurrentLiabilities
otherLiabilities
capitalLeaseObligations
totalLiabilities
preferredStock
commonStock
retainedEarnings
acc

The above output was used to construct the Variable Index file.

In [ ]:
# Load variable index
variable_index = pd.read_excel('Variable Index.xlsx')
variable_index

In [1]:
# Get disallowed variables
disallowed_variables = list(variable_index[variable_index['Variable Type'] == 'Disallowed']['column_name'])

# Drop observations missing anything else
for col in all_data_with_NLP_features.columns:
    if col not in disallowed_variables:
        all_data_with_NLP_features_drop = all_data_with_NLP_features[all_data_with_NLP_features[col].notnull()]
        if len(all_data_with_NLP_features_drop) < len(all_data_with_NLP_features):
            print(col, 'missing - dropped ', len(all_data_with_NLP_features) - len(all_data_with_NLP_features_drop))
        all_data_with_NLP_features = all_data_with_NLP_features_drop
all_data_with_NLP_features

NameError: name 'all_data_with_NLP_features' is not defined

In [55]:
# min and max of fixed_quarter_date
print('min of fixed_quarter_date:', min(all_data_with_NLP_features['fixed_quarter_date']))
print('max of fixed_quarter_date:', max(all_data_with_NLP_features['fixed_quarter_date']))

min of fixed_quarter_date: 2011-01-01
max of fixed_quarter_date: 2016-10-01


## Stratified Random Sampling on Rating

In [56]:
# Set seed to 2
import random
random.seed(2)
# If column 'train_test_80_20' exists, remove it
if 'train_test_80_20' in all_data_with_NLP_features.columns:
    all_data_with_NLP_features = all_data_with_NLP_features.drop('train_test_80_20', axis = 1)
# Perform stratified random sampling to create new column train_test_80_20
# Split into dataframes by rating
list_of_rating_dfs = [all_data_with_NLP_features[all_data_with_NLP_features['Rating'] == rating] for rating in all_data_with_NLP_features['Rating'].unique()]
rating_dfs_with_split = []
# Iterate over dfs
for rating_df in list_of_rating_dfs:
    # Create column train_test_80_20 with 80% probability of train
    # Create list of random uniforms of length of rating_df
    rand_uniforms = [random.uniform(0, 1) for _ in range(rating_df.shape[0])]
    # Train and test based off of uniforms
    rating_df['train_test_80_20'] = ['train' if x < 0.8 else 'test' for x in rand_uniforms]
    rating_dfs_with_split.append(rating_df)
# Concatenate the dataframes
all_data_with_NLP_features_train_test = pd.concat(rating_dfs_with_split)
print('value counts of train_test_80_20 by rating')
print(all_data_with_NLP_features_train_test[['Rating', 'train_test_80_20']].value_counts().sort_index())
all_data_with_NLP_features_train_test


value counts of train_test_80_20 by rating
Rating  train_test_80_20
A       test                 190
        train                752
AA      test                  38
        train                155
AAA     test                  25
        train                 77
B       test                 123
        train                529
BB      test                 251
        train               1014
BBB     test                 275
        train               1166
C       test                   2
        train                 14
CC      test                   2
        train                  4
CCC     test                  21
        train                 77
D       test                   4
        train                  5
Name: count, dtype: int64


C:\Users\ijyli\AppData\Local\Temp\ipykernel_11016\2733271515.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['train_test_80_20'] = ['train' if x < 0.8 else 'test' for x in rand_uniforms]
C:\Users\ijyli\AppData\Local\Temp\ipykernel_11016\2733271515.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['train_test_80_20'] = ['train' if x < 0.8 else 'test' for x in rand_uniforms]
C:\Users\ijyli\AppData\Local\Temp\ipykernel_11016\2733271515.py:17: SettingWithCopyWarning: 
A value is tryi

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,...,Active,Passive,Ovrst,Undrst,PN,SW,AP,OU,TONE1,train_test_80_20
1,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,495.0,186.0,364.0,131.0,5.518519,15.261905,2.661290,2.778626,3.188264,test
2,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,656.0,199.0,465.0,152.0,5.348485,15.934783,3.296482,3.059211,3.681858,test
3,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,AA,2015-05-28,AA,2014-05-27,2015-05-28,...,591.0,208.0,468.0,151.0,3.927711,8.113636,2.841346,3.099338,1.307366,train
4,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,AA,2015-08-25,AA,2015-05-28,2015-08-25,...,565.0,214.0,415.0,135.0,5.250000,9.142857,2.640187,3.074074,2.025933,train
5,AAPL,2015-10-01,2015-07-21,AA,2015-08-25,AA,2016-05-20,AA,2015-06-02,2016-05-20,...,565.0,219.0,449.0,148.0,4.209877,10.442857,2.579909,3.033784,1.815531,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,MHO,2013-10-01,2013-07-25,CC,2013-05-07,CC,2013-11-06,CCC,2012-11-29,2013-11-06,...,295.0,122.0,180.0,82.0,7.153846,10.225000,2.418033,2.195122,2.140598,train
3749,MHO,2014-01-01,2013-10-24,CC,2013-11-06,CCC,2014-07-31,CC,2013-05-07,2014-07-31,...,462.0,152.0,294.0,166.0,4.516129,6.771084,3.039474,1.771084,-0.029287,train
3750,MHO,2014-04-01,2014-01-29,CC,2013-11-06,CCC,2014-07-31,CC,2013-05-07,2014-07-31,...,444.0,136.0,325.0,158.0,3.835821,7.078947,3.264706,2.056962,0.084888,train
3751,MHO,2014-07-01,2014-04-24,CC,2013-11-06,CCC,2014-07-31,CC,2013-05-07,2014-07-31,...,329.0,127.0,194.0,103.0,3.703704,6.944444,2.590551,1.883495,-0.319794,test


## Fragment and store on GitHub

In [57]:
def split_df(df, dataset_name, out_folder, num_pieces):
    '''
    Splits dataframes into num_pieces and saves them as parquet files in out_folder. Reduces file size to comply with GitHub limits.
    '''
    # Tracking total length of pieces
    total_len_pieces = 0
    # Create out_folder if it does not exist
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    # Delete previous pieces, all contents of out_folder
    for filename in os.listdir(out_folder):
        file_path = os.path.join(out_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    # Save pieces
    for i in range(num_pieces):
        # start index for piece rows
        start_index = i * len(df) // num_pieces
        # end index for piece rows
        end_index = (i + 1) * len(df) // num_pieces
        # get piece
        piece = df[start_index:end_index]
        piece.to_parquet(out_folder + '/' + dataset_name + '_piece_' + str(i) + '.parquet', index=False)
        #print(len(piece))
        total_len_pieces += len(piece)
    # check total piece length and length of vit_embeddings_df
    print('length check passed')
    print(total_len_pieces == len(df))

In [58]:
split_df(all_data_with_NLP_features, 'All_Data_with_NLP_Features', '../../../Data/All_Data/All_Data_with_NLP_Features', 10)

length check passed
True
